In [ ]:
#|default_exp mic

# Mic streaming and recording

In [ ]:
#| export
import sys 
import queue

import zmq
import fire
import sounddevice as sd

In [ ]:
#| export 
from asr_teach.utils import SAMPLE_RATE, DEVICE, BLOCK_DURATION, DTYPE
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

In [ ]:
#| export

class MicStream:
    def __init__(self,
                 sample_rate: float = None,
                 device: int = 0,
                 block_duration: int = 2500,
                 dtype: str = DTYPE,
                 addr: str = ADDR,
                 port: int = PORT,
                 protocol: str = PROTOCOL,
                 context=None,
                 q=None):
        
        # create the socket
        context = context or zmq.Context()
        socket = context.socket(zmq.PUSH)
        socket.bind(f'{protocol}://{addr}:{port}')
        self.socket = socket
        
        # queue to send over the data
        self.q = q or queue.Queue()

        # set the sample rate
        self.sample_rate = sample_rate or sd.query_devices(device, 'input')['default_samplerate']
        self.num_samples = int(sample_rate * block_duration)

        def callback(indata, frames, time, status):
            '''Places audio data on the queue for sending over zmq port.
            '''
            if any(indata):
                self.q.put(indata)
        
        # create the microphone streaming object
        self.stream = sd.InputStream(device=device,
                                     channels=1,
                                     callback=callback,
                                     dtype=dtype,
                                     blocksize=self.num_samples // 1000,
                                     samplerate=sample_rate)
        
    def send_audio(self, indata, flags=0, copy=True, track=False):
        '''Sends the first message with array shape and data type.
        '''
        md = {'dtype': str(indata.dtype),
              'shape': indata.shape}
        self.socket.send_json(md, zmq.SNDMORE)
        # sends the array data
        self.socket.send(indata, flags, copy=copy, track=track)
        
        
    def run(self):
        '''Streams audio until the user stops or interrupts the process.
        '''
        # start streaming with context manager
        self.stream.start()
        while True:
            try:
                if not self.q.empty():
                    self.send_audio(self.q.get())
            except KeyboardInterrupt:
                print('User keyboard interrupt.')
                print('stopping stream')
                self.stream.close()
                break
                print('stream stopped')
            except Exception as e:
                print(f'Exception: {e}')
                raise
            
#         try:
#             # start streaming with context manager
#             with self.stream:
#                 sd.wait(self.num_samples)
#         except KeyboardInterrupt:
#             print('User set keyboard interrupt.')
#             sys.exit(0)
#         except Exception as e:
#             print(f'Exception: {e}')
#             raise

In [ ]:
#| export

def stream(sample_rate: float = SAMPLE_RATE,
           device: int = DEVICE,
           block_duration: int = BLOCK_DURATION,
           **kwargs):
    '''Entrypoint for live microphone streaming over a port.
    '''
    # create and start the microphone stream
    mic = MicStream(sample_rate, device, block_duration, **kwargs)
    mic.run()